# Nesterov et al, arXiv:2011.10011
In this notebook we want to capture the basic effect behind the two-qubit gate for fluxonia described in the paper by Nesterov et al(2020) "Proposal for entangling gates on fluxonium qubits via a two-photon transition"

In [17]:
import numpy as np
import sys
import qutip as qtp 
import matplotlib.pyplot as plt
from scipy import constants
import pysqkit
from pysqkit.util.linalg import get_mat_elem

The gate is based on the following driven Hamiltonian of two-capacitively coupled fluxonium qubits

$$\hat{H} = \hat{H}^{(0)}_A + \hat{H}^{(0)}_B + \hat{V} + \hat{H}_{drive}$$

$$\hat{H}^{(0)}_A = 4 E_{C,A} \cdot \hat{n}^2_A + \frac{1}{2}E_{L,A}\cdot\hat{\varphi}^2_A - E_{J,A}\cos\left(\hat{\varphi}_A - \phi_{ext,A}\right)$$

$$\hat{H}^{(0)}_B = 4 E_{C,B} \cdot \hat{n}^2_B + \frac{1}{2}E_{L,B}\cdot\hat{\varphi}^2_B - E_{J,B}\cos\left(\hat{\varphi}_B - \phi_{ext,B}\right)$$

$$\hat{V} = J_C \cdot \hat{n}_A \cdot \hat{n}_B$$

$$\hat{H}_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \cdot \left( \eta_A \cdot \hat{n}_A + \eta_B \cdot \hat{n}_B \right)   \text{  with  } f\left( t \right) = f_0 \cdot \dfrac{f\left( t \right)}{f_0} $$

## 1. Single fluxonia
We take parameters as in Table 1 of the paper

In [18]:
# Values in GHz (divided by h)
#Fluxonium A
ec_a = 1.0
el_a = 1.5
ej_a = 3.8 
flux_a = 1/2
# Fluxonium B 
ec_b = 1.0
el_b = 0.9
ej_b = 3.0
flux_b = 1/2

n_fock = 100

Let us first reproduce the other parameters in Table 1

In [19]:
flx_a = pysqkit.qubits.Fluxonium('A', ec_a, el_a, ej_a, flux_a, dim_hilbert=n_fock)
flx_b = pysqkit.qubits.Fluxonium('B', ec_b, el_b, ej_b, flux_b, dim_hilbert=n_fock)

In [20]:
levels = 4
energies_a, eig_states_a = flx_a.eig_states(levels)
energies_b, eig_states_b = flx_b.eig_states(levels)
for k in range(0, 3):
    print('------')
    print('f_' + str(k) + str(k + 1) + '^A = ' + str(energies_a[k + 1] - energies_a[k]) + ' GHz' )
    print('f_' + str(k) + str(k + 1) + '^B = ' + str(energies_b[k + 1] - energies_b[k]) + ' GHz' )
print('------')

------
f_01^A = 1.1516938223021964 GHz
f_01^B = 0.8488266605313166 GHz
------
f_12^A = 3.280479803830147 GHz
f_12^B = 2.9286387379969865 GHz
------
f_23^A = 3.2531360533441145 GHz
f_23^B = 2.6834970613115905 GHz
------


Thus, we correctly recover the frequencies Table 1 in the paper. Notice that the eigenenergies are also divided by $h$ and so are intended as frequencies and not as angular velocities. <br>
Let us also check the charge matrix elements in order to complete Table 1

In [21]:
for k in range(0, 2):
    print('------')
    print('|n_' + str(k) + str(k + 1) + '^A| = ' + \
          str(np.abs(get_mat_elem(flx_a.charge_op(), eig_states_a[k], eig_states_a[k + 1]))))
    print('|n_' + str(k) + str(k + 1) + '^B| = ' + \
          str(np.abs(get_mat_elem(flx_b.charge_op(), eig_states_b[k], eig_states_b[k + 1]))))
print('------')
print('|n_' + str(0) + str(3) + '^A| = '  + \
      str(np.abs(get_mat_elem(flx_a.charge_op(), eig_states_a[0], eig_states_a[3]))))
print('|n_' + str(0) + str(3) + '^B| = '  + \
      str(np.abs(get_mat_elem(flx_b.charge_op(), eig_states_b[0], eig_states_b[3]))))

------
|n_01^A| = 0.24866384335580258
|n_01^B| = 0.20722487336294326
------
|n_12^A| = 0.6084791377921823
|n_12^B| = 0.5665046936475003
------
|n_03^A| = 0.2601795875970435
|n_03^B| = 0.277316590424727


We thus recover all the results in Table 1. This could be used as a unit test. <br>
To change the basis to the diagonal basis you can use

In [ ]:
#flx_a.diagonalize_basis(levels)
#flx_b.diagonalize_basis(levels)
# If you run this twice it gives an error, so this has to be fixed in the source code

## 2. Coupled fluxonia

As a preliminary study, I want to plot  